# CSI 4142 Deliverable 3

## Imports & Constants

In [215]:
import numpy as np
import pandas as pd
from os import path

import calendar

# Constants
FORCE_NEW_CSV = True

MONTH_DIM_CSV_FILE = ".\\dim\\month_dim.csv"

FACT_TABLE_DATA_FILE = ".\\data\\fact_data.csv"
FACT_TABLE_DIM_FILE = ".\\dim\\fact_dim.csv"


COUNTRY_DATA_CSV_FILE = ".\\data\\location_data.csv"
COUNTRY_DIM_CSV_FILE  = ".\\dim\\location_dim.csv"

POPULATION_DATA_CSV_FILE = ".\\data\\population_data.csv"
POPULATION_DIM_CSV_FILE  = ".\\dim\\population_dim.csv"

QOL_DATA_CSV_FILE = ".\\data\\qol_data.csv"
QOL_DIM_CSV_FILE  = ".\\dim\\qol_dim.csv"

HEALTH_DATA_CSV_FILE = ".\\data\\health_data.csv"
HEALTH_DIM_CSV_FILE  = ".\\dim\\health_dim.csv"

EVENT_DATA_CSV_FILE = ".\\data\\event_data.csv"
EVENT_DIM_CSV_FILE = ".\\dim\\event_dim.csv"

EDUCATION_DATA_CSV_FILE = ".\\data\\education_data.csv"
EDUCATION_DIM_CSV_FILE = ".\\dim\\education_dim.csv"

YEARS_COLS = [f"{2005 + x} [YR{2005 + x}]" for x in range(16)] #Build 

YEARS_COLS.insert(0,"Series Name")



## Save function

In [216]:
def save_to_csv(save_to, df):
    if (path.exists(save_to) and not FORCE_NEW_CSV):
        print("Already Have This Dimensions' CSV")
    else:
        df.to_csv(save_to, index=False)
    

## Preprocess Month Dim

In [217]:
years = [f"{2005 + x}" for x in range(16)]
months = [calendar.month_name[mon] for mon in range(1,13)]
rows = []
for year in years:
    for i, month in enumerate(months):
        month_row = {}
        month_row["month_id"] = f"{year}_{month}"
        month_row["name"] = month
        month_row["quarter"] = i // 3 + 1
        month_row["year"] = year
        month_row["decade"] = int(year) // 10 * 10
        rows.append(month_row)
month_df = pd.DataFrame.from_dict(rows,orient='columns')
month_df.insert(0, "month_key", np.arange(start= 1, stop = len(month_df) + 1 ))
month_df.tail()

,month_key,month_id,name,quarter,year,decade
187,188,2020_August,August,3,2020,2020
188,189,2020_September,September,3,2020,2020
189,190,2020_October,October,4,2020,2020
190,191,2020_November,November,4,2020,2020
191,192,2020_December,December,4,2020,2020


In [218]:
save_to_csv(MONTH_DIM_CSV_FILE, month_df)

## Preprocess Country Dim

In [219]:
country_extra_fields = {
  "Canada": {
    "currency": "CAD",
    "continent": "North America",
    "capital": "Ottawa",
    "region": "North America"

  },
  "United States": {
    "currency": "USD",
    "continent": "North America",
    "capital": "Washington",
    "region": "North America"

  },
  "Mexico": {
    "currency": "Peso",
    "continent": "North America",
    "capital": "Mexico City",
    "region": "North America"

  },
  "Australia": {
    "currency": "AUD",
    "continent": "Australia",
    "capital": "Canberra",
    "region": "Australia"
  },
  "Chad": {
    "currency": "CFA franc",
    "continent": "Africa",
    "capital": "N'Djamena",
    "region": "Central Africa"
  },
    "Germany": {
    "currency": "Euro",
    "continent": "Europe",
    "capital": "Berlin",
    "region": "Central Europe"
  },
  "Kenya": {
    "currency": "Shilling",
    "continent": "Africa",
    "capital": "Nairobi",
    "region": "East Africa"
    
  },
    "Switzerland": {
    "currency": "CHF",
    "continent": "Europe",
    "capital": "Bern",
    "region": "Central Europe"

  },
  "Venezuela, RB": {
    "currency": "Bolivar",
    "continent": "South America",
    "capital": "Caracas",
    "region": "South America"
  }
}

country_data = pd.read_csv(COUNTRY_DATA_CSV_FILE)

countries = set(country_data["Country Name"])
all_series = set(country_data["Series Name"])
rows = []
for country in countries:
  cun_df = country_data[country_data["Country Name"] == country]
  iso = cun_df["Country Code"].values[0]
  c = cun_df[YEARS_COLS].set_index("Series Name").T
  for ind, measure in c.iterrows():
    row = {}    
    row["name"] = country
    row["iso"] = iso
    row["year"] = ind[:4]
    row["region"] = country_extra_fields[country]["region"]
    row["currency"] = country_extra_fields[country]["currency"]
    row["capital"] = country_extra_fields[country]["capital"]
    row["continent"] = country_extra_fields[country]["continent"]
    for series in all_series:
      row[series] = measure[series] if measure[series] != ".." else -1
    rows.append(row)

country_dim_df = pd.DataFrame.from_dict(rows, orient='columns')
country_dim_df.insert(0, "country_id", np.arange(start=0, stop=len(country_dim_df)))
country_dim_df.insert(0, "country_key", np.arange(start=100, stop=len(country_dim_df) + 100))
country_dim_df.head()


,country_key,country_id,name,iso,year,region,currency,capital,continent,"Age dependency ratio, young",...,"Fertility rate, total (births per woman)",Human capital index (HCI) (scale 0-1),"Death rate, crude (per 1,000 people)","Age dependency ratio, old","Birth rate, crude (per 1,000 people)",Poverty headcount ratio at national poverty line (% of population),"Population, male (% of total population)","Labor force, total",Age dependency ratio (% of working-age population),"GNI per capita, Atlas method (current US$)"
0,100,0,Kenya,KEN,2005,East Africa,Shilling,Nairobi,Africa,81.69459705,...,4.843,-1,10.544,3.861197694,38.366,46.8,49.61659278,14988416,85.55579475,510
1,101,1,Kenya,KEN,2006,East Africa,Shilling,Nairobi,Africa,81.10808913,...,4.767,-1,9.848,3.797743512,37.89,-1,49.6211311,15477612,84.90583264,580
2,102,2,Kenya,KEN,2007,East Africa,Shilling,Nairobi,Africa,80.80659069,...,4.682,-1,9.158,3.724661891,37.33,-1,49.62808545,15959067,84.53125258,700
3,103,3,Kenya,KEN,2008,East Africa,Shilling,Nairobi,Africa,80.6081537,...,4.587,-1,8.516,3.651025421,36.678,-1,49.63628856,16446062,84.25917912,820
4,104,4,Kenya,KEN,2009,East Africa,Shilling,Nairobi,Africa,80.22546532,...,4.482,-1,7.948,3.586838305,35.942,-1,49.64437749,16962552,83.81230362,820


In [220]:
save_to_csv(COUNTRY_DIM_CSV_FILE, country_dim_df)

## Preprocess Func

In [221]:
def basic_preprocess(csvfile, key, defualt_val = -1, offset = 100):
    data = pd.read_csv(csvfile)
    countries = set(data["Country Name"])
    all_series = set(data["Series Name"])
    rows = []
    for country in countries:
        country_df = data[data["Country Name"] == country]
        iso = country_df["Country Code"].values[0]
        country_df = country_df[YEARS_COLS].set_index("Series Name").T
        for ind, measure in country_df.iterrows():
            row = {}   
            row["country"] = country
            row["iso"] = iso
            row["year"] = ind[:4]
            for series in all_series:
                row[series] = measure[series] if measure[series] != ".." else defualt_val
            rows.append(row)

    ret_df = pd.DataFrame.from_dict(rows, orient='columns')
    ret_df.insert(0, f"{key}_id", np.arange(0, stop=len(ret_df)))
    ret_df.insert(0, f"{key}_key", np.arange(start=offset, stop=len(ret_df) + offset))
    ret_df.fillna(-1, inplace=True)
    return ret_df

## Preprocess Population Dim


In [222]:
population_df = basic_preprocess(POPULATION_DATA_CSV_FILE, "population")
save_to_csv(POPULATION_DIM_CSV_FILE, population_df)
population_df.head()

,population_key,population_id,country,iso,year,Population ages 65 and above (% of total population),Population ages 15-64 (% of total population),"Population, female (% of total population)","Survival to age 65, male (% of cohort)","Population ages 65 and above, male (% of male population)",...,"Population ages 15-64, female (% of female population)",Population growth (annual %),Population ages 0-14 (% of total population),Urban population growth (annual %),"Population ages 65 and above, female (% of female population)","Population ages 0-14, female (% of female population)","Suicide mortality rate, male (per 100,000 male population)",Sex ratio at birth (male births per female births),"Life expectancy at birth, total (years)","Suicide mortality rate (per 100,000 population)"
0,100,0,Kenya,KEN,2005,2.08088241618167,53.8921463119553,50.3834072215957,41.9801546,1.74491275766389,...,54.1290024733821,2.73924614871763,44.026971271863,4.43755369089764,2.4117331118827,43.4592644147353,9.2,-1.00,54.732,6
1,101,1,Kenya,KEN,2006,2.0538801089526,54.0815829187432,50.3788689002128,44.9244618,1.7207637411633,...,54.3315024494969,2.75791743051646,43.8645369723042,4.45055664191255,2.38204348990571,43.2864540605974,9.1,-1.00,56.093,6
2,102,2,Kenya,KEN,2007,2.018445,54.191363,50.371915,47.868769,1.688426,...,54.455519,2.768549,43.790192,4.4553,2.343662,43.200818,9.3,1.03,57.463,6.3
3,103,3,Kenya,KEN,2008,1.981462,54.271379,50.363711,49.301163,1.654071,...,54.548784,2.767256,43.747159,4.447969,2.304194,43.147023,9.1,1.03,58.76,6.2
4,104,4,Kenya,KEN,2009,1.951359,54.403323,50.355623,50.733557,1.625062,...,54.689483,2.750854,43.645318,4.416731,2.273116,43.037402,8.7,1.03,59.932,6.0


## Preprocessing Health Dim

In [223]:
health_df = basic_preprocess(HEALTH_DATA_CSV_FILE, "health", offset=135)
save_to_csv(HEALTH_DIM_CSV_FILE, health_df)
health_df.head()

,health_key,health_id,country,iso,year,Risk of catastrophic expenditure for surgical care (% of people at risk),"Immunization, BCG (% of one-year-old children)","Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%)",Antiretroviral therapy coverage (% of people living with HIV),Lifetime risk of maternal death (%),...,"Hospital beds (per 1,000 people)",Newborns protected against tetanus (%),Prevalence of undernourishment (% of population),"Immunization, DPT (% of children ages 12-23 months)","Incidence of tuberculosis (per 100,000 people)","Mortality rate, infant (per 1,000 live births)",Births attended by skilled health staff (% of total),Domestic general government health expenditure (% of GDP),Tuberculosis treatment success rate (% of new cases),"Immunization, measles (% of children ages 12-23 months)"
0,135,0,Kenya,KEN,2005,60.2,85,22.2,4,2.95788757463242,...,-1,73,28.5,76,630,48,-1,1.48269117,81,69
1,136,1,Kenya,KEN,2006,54.6,92,22.3,9,2.77944497600631,...,1.4,74,26.1,80,646,45.7,-1,1.45845628,83,77
2,137,2,Kenya,KEN,2007,49.5,92,22.7,13,2.60044345863232,...,-1,74,26.9,81,618,43.7,-1,1.55638492,83,80
3,138,3,Kenya,KEN,2008,47.7,98,22.7,18,2.43283960474894,...,-1,78,26.6,88,564,41.4,-1,1.63804531,84,90
4,139,4,Kenya,KEN,2009,45.3,99,22.4,25,2.21745886675819,...,-1,78,24.1,88,566,40.4,43.8,1.68060637,84,88


## Preprocessing Education Dim

In [224]:
education_df = basic_preprocess(EDUCATION_DATA_CSV_FILE, "education", offset=215)
save_to_csv(EDUCATION_DIM_CSV_FILE, education_df)
education_df.head()

,education_key,education_id,country,iso,year,"Primary completion rate, total (% of relevant age group)","School enrollment, tertiary (% gross)","School enrollment, secondary (% gross)","School enrollment, secondary, male (% gross)","Primary completion rate, male (% of relevant age group)","School enrollment, primary, female (% gross)","School enrollment, secondary, female (% gross)","Public spending on education, total (% of GDP)","School enrollment, primary (% gross)","School enrollment, primary, male (% gross)","Primary completion rate, female (% of relevant age group)"
0,215,0,Kenya,KEN,2005,86.942497253418,2.93320989608765,46.9905700683594,48.3319892883301,88.2794418334961,97.1405487060547,45.6516799926758,7.3356499671936,99.3462905883789,101.536567687988,85.6027069091797
1,216,1,Kenya,KEN,2006,-1,-1,48.7714500427246,50.6877708435059,-1,94.8505783081055,46.8591117858887,7.04983997344971,96.347541809082,97.8325271606445,-1
2,217,2,Kenya,KEN,2007,-1,-1,50.7726707458496,54.4201698303223,-1,101.510322570801,47.1313018798828,-1,102.365493774414,103.212959289551,-1
3,218,3,Kenya,KEN,2008,-1,-1,56.5935211181641,59.3309516906738,-1,101.154037475586,53.8584594726563,-1,102.284797668457,103.404220581055,-1
4,219,4,Kenya,KEN,2009,-1,3.99340009689331,56.760368347168,59.8581085205078,-1,102.748573303223,53.6618194580078,-1,103.914497375488,105.067558288574,-1


## Preprocess Event Dim

In [225]:
def month_diff(start_year, start_mon, end_year, end_mon):
    start_ind = (start_year - 2005) * 12 + start_mon
    diff = (end_year - 2005) * 12 + end_mon - start_ind
    return  diff

In [226]:
data = pd.read_csv(EVENT_DATA_CSV_FILE)
rows = []
cols = set(data.columns) - set(["Start Year","Start Month","End Year","End Month", "Year"])
for _, event in data.iterrows():
    if (np.isnan(event["Start Year"]) or np.isnan(event["Start Month"]) or np.isnan(event["End Year"]) or np.isnan(event["End Month"])): continue
    start_year = int(event["Start Year"])
    start_mon = int(event["Start Month"])
    end_year = int(event["End Year"])
    end_mon = int(event["End Month"])
    diff = month_diff(start_year, start_mon, end_year, end_mon) + 1
    
    for mon_num in range(diff): 
        row = {}
        for col in cols:
            dt = data[col].dtype
            row[col] = event[col] if(not pd.isna(event[col])) else (-1 if dt == int or dt == float else "N/A")
        row["date"] = f"{start_year}_{calendar.month_name[start_mon]}"
        rows.append(row)
        if ((start_mon + 1) % 13 == 0):
            start_mon = 1
            start_year += 1
        else:
            start_mon +=1
event_df = pd.DataFrame.from_dict(rows, orient='columns')
event_df.insert(0, "event_id", np.arange(start=0, stop=len(event_df)))
event_df.insert(0, "event_key", np.arange(start=90, stop=len(event_df) + 90))
event_df.fillna(-1, inplace=True)
save_to_csv(EVENT_DIM_CSV_FILE, event_df)
event_df.tail()

,event_key,event_id,Event Name,ISO,Disaster Type,Disaster Subtype,Total Affected,Disaster Subgroup,Dis No,"Total Damages, Adjusted ('000 US$)",Disaster Group,CPI,Total Deaths,date
1418,1508,1418,August Complex fire,USA,Wildfire,Forest fire,-1.0,Climatological,2020-0441-USA,11516761.0,Natural,95.512967,32.0,2020_August
1419,1509,1419,August Complex fire,USA,Wildfire,Forest fire,-1.0,Climatological,2020-0441-USA,11516761.0,Natural,95.512967,32.0,2020_September
1420,1510,1420,August Complex fire,USA,Wildfire,Forest fire,-1.0,Climatological,2020-0441-USA,11516761.0,Natural,95.512967,32.0,2020_October
1421,1511,1421,Migrants,VEN,Transport accident,Water,-1.0,Technological,2020-0549-VEN,-1.0,Technological,95.512967,28.0,2020_December
1422,1512,1422,N/A,VEN,Flood,N/A,3690.0,Hydrological,2020-0498-VEN,31409.0,Natural,95.512967,3.0,2020_November


## Preprocessing QOL Dim

In [227]:
qol_df = basic_preprocess(QOL_DATA_CSV_FILE, "qol", offset=405)
save_to_csv(QOL_DIM_CSV_FILE, qol_df)
qol_df.tail()

,qol_key,qol_id,country,iso,year,"Unemployment, total (% of total labor force)",People using safely managed drinking water services (% of population),"Unemployment, male (% of male labor force)",Maternal leave benefits (% of wages paid in covered period),People using at least basic sanitation services (% of population),People using safely managed sanitation services (% of population),Prevalence of hypertension (% of adults ages 30-79),Proportion of population spending more than 10% of household consumption or income on out-of-pocket health care expenditure (%),"Unemployment, female (% of female labor force)",Treatment for hypertension (% of adults ages 30-79 with hypertension),People using at least basic drinking water services (% of population),Risk of impoverishing expenditure for surgical care (% of people at risk)
139,544,139,Switzerland,CHE,2016,4.920000076,94.24815394,4.84100008,-1,99.89829607,99.63594918,23.4,-1,5.011000156,54.8,100.0000029,0
140,545,140,Switzerland,CHE,2017,4.800000191,94.24815119,4.572000027,80,99.89987358,99.63774647,22.9,-1,5.063000202,55.3,100,0
141,546,141,Switzerland,CHE,2018,4.710000038,94.24815119,4.366000175,-1,99.89987362,99.63810616,22.4,-1,5.105000019,55.7,100,0
142,547,142,Switzerland,CHE,2019,4.389999866,94.24815119,4.109000206,-1,99.89987367,99.63862569,21.9,-1,4.709000111,56.1,100,0
143,548,143,Switzerland,CHE,2020,4.820000172,94.24815385,4.660999775,-1,99.89987655,99.65155911,-1,-1,5.000999928,-1,100.0000028,0


## Building Fact Table

In [228]:
fact_table_rows = []

fact_table_data = pd.read_csv(FACT_TABLE_DATA_FILE)

fact_table_measures = ["Human Development Index", "Quality of Life","Education Index","GNI Per Capita","Global Peace Index"]

for _, event in event_df.iterrows():
    row= {}
    year =  event["date"][:4]
    
    row["event_key"] = event["event_id"]
    row["country_key"] = country_dim_df[(country_dim_df["iso"] == event["ISO"]) & (country_dim_df["year"] == year)]["country_id"].values[0]
    row["date_key"] = month_df[month_df["month_id"] == event["date"]]["month_id"].values[0]
    row["education_key"] = education_df[(education_df["iso"] == event["ISO"]) & (education_df["year"] ==  year )]["education_id"].values[0]
    row["population_key"] = population_df[(population_df["iso"] == event["ISO"]) & (population_df["year"] ==  year )]["population_id"].values[0]
    row["health_key"] = health_df[(health_df["iso"] == event["ISO"]) & (health_df["year"] ==  year )]["health_id"].values[0]
    row["qol_key"] = qol_df[(qol_df["iso"] == event["ISO"]) & (qol_df["year"] ==  year )]["qol_id"].values[0]
    for fact_measure in fact_table_measures:
        val = fact_table_data[(fact_table_data["Country Code"] == event["ISO"]) & (fact_table_data["Measure"] == fact_measure)][year].values[0]
        row[fact_measure] = val if not np.isnan(val) else -1
    fact_table_rows.append(row)
fact_table_df = pd.DataFrame.from_dict(fact_table_rows,orient='columns')
fact_table_df.insert(0, "fact_id", np.arange(start=0, stop=len(fact_table_df)))
fact_table_df.head()

,fact_id,event_key,country_key,date_key,education_key,population_key,health_key,qol_key,Human Development Index,Quality of Life,Education Index,GNI Per Capita,Global Peace Index
0,0,0,80,2005_January,80,80,80,80,0.906,-1.0,0.873,31440.0,-1.0
1,1,1,80,2005_January,80,80,80,80,0.906,-1.0,0.873,31440.0,-1.0
2,2,2,96,2005_January,96,96,96,96,0.908,-1.0,0.893,32060.0,-1.0
3,3,3,16,2005_February,16,16,16,16,0.900,-1.0,0.862,44550.0,-1.0
4,4,4,16,2005_January,16,16,16,16,0.900,-1.0,0.862,44550.0,-1.0


## Replacing with IDs with surrogate keys

In [229]:
for i, fact_row in fact_table_df.iterrows():
    fact_table_df.at[i,"event_key"] = event_df[event_df["event_id"]==fact_row["event_key"]]["event_key"].values[0]
    fact_table_df.at[i,"country_key"] = country_dim_df[country_dim_df["country_id"] == fact_row["country_key"]]["country_key"].values[0]
    fact_table_df.at[i,"date_key"] = month_df[month_df["month_id"]==fact_row["date_key"]]["month_key"].values[0]
    fact_table_df.at[i,"education_key"] = education_df[education_df["education_id"]==fact_row["education_key"]]["education_key"].values[0]
    fact_table_df.at[i,"population_key"] = population_df[population_df["population_id"]==fact_row["population_key"]]["population_key"].values[0]
    fact_table_df.at[i,"health_key"] = health_df[health_df["health_id"]==fact_row["health_key"]]["health_key"].values[0]
    fact_table_df.at[i,"qol_key"] = qol_df[qol_df["qol_id"]==fact_row["qol_key"]]["qol_key"].values[0]
    
fact_table_df.head()

,fact_id,event_key,country_key,date_key,education_key,population_key,health_key,qol_key,Human Development Index,Quality of Life,Education Index,GNI Per Capita,Global Peace Index
0,0,90,180,1,295,180,215,485,0.906,-1.0,0.873,31440.0,-1.0
1,1,91,180,1,295,180,215,485,0.906,-1.0,0.873,31440.0,-1.0
2,2,92,196,1,311,196,231,501,0.908,-1.0,0.893,32060.0,-1.0
3,3,93,116,2,231,116,151,421,0.900,-1.0,0.862,44550.0,-1.0
4,4,94,116,1,231,116,151,421,0.900,-1.0,0.862,44550.0,-1.0


In [230]:
save_to_csv(FACT_TABLE_DIM_FILE, fact_table_df)